<a href="https://colab.research.google.com/github/Blane187/AI-Song-Cover-RVC/blob/main/Inference_NO_UI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![ko-fi](https://ko-fi.com/img/githubbutton_sm.svg)](https://ko-fi.com/R6R7AH1FA)
<a href="https://trakteer.id/ardha27">
    <img src="https://cdn.trakteer.id/images/embed/trbtn-red-1.png" alt="Trakteer" height="35">
</a>

In [ ]:
#@title Mount Drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Unzip File From Drive

!unzip -n /content/drive/MyDrive/assets/KiwKiw.zip -d /content
%cd /content/KiwKiw
!pip install -r requirements.txt -q
!pip install mega.py
!pip install yt_dlp
!pip install ffmpeg
!mkdir youtubeaudio

In [ ]:
#@title Download The Model
from mega import Mega
import os
import IPython.display as clear_output, display
MODEL = "Ardha"  #@param {type:"string"}
MODELEPOCH = ''
url='https://huggingface.co/ardha27/RVC/resolve/main/Ardha.zip?download=true'#@param {type:"string"}
MODELZIP = MODEL + '.zip'
modelname_path='/content/zips/'+MODELZIP
if url != '':
  !mkdir -p /content/KiwKiw/logs/{MODEL}
  !mkdir -p /content/zips/
  if "drive.google.com" in url:
    !gdown $url --fuzzy -O $modelname_path
  elif "mega.nz" in url:
    m = Mega()
    m.download_url(url, '/content/zips')
  else:
    !wget $url -O /content/zips/{MODELZIP}
  for filename in os.listdir("/content/zips"):
    if filename.endswith(".zip"):
      zip_file = os.path.join("/content/zips", filename)
      shutil.unpack_archive(zip_file, "/content/unzips", 'zip')
  #Move model into logs folder
  for root, dirs, files in os.walk('/content/unzips'):
    for file in files:
      if "G_" in file:
        MODELEPOCH = file.split("G_")[1].split(".")[0]
    if MODELEPOCH == '':
      MODELEPOCH = '404'
    for file in files:
      file_path = os.path.join(root, file)
      if file.endswith(".npy") or file.endswith(".index"):
        !mv {file_path} /content/KiwKiw/logs/{MODEL}/
      elif "G_" not in file and "D_" not in file and file.endswith(".pth"):
        !mv {file_path} /content/KiwKiw/weights/{MODEL}.pth

  !rm -r /content/unzips/
  !rm -r /content/zips/
  clear_output()
  display(success)
else:
  print("URL cannot be left empty. If you don't want to download a model now, just skip this step.")

In [ ]:
#@title INFER NO UI
import os
import IPython.display as ipd
%cd  /content/KiwKiw
model_path = "/content/KiwKiw/assets/weights/My-Voice.pth"#@param {type:"string"}
index_path = "/content/KiwKiw/logs/My-Voice/added_IVF439_Flat_nprobe_1_My-Voice_v2.index"#@param {type:"string"}
from colorama import Fore
print(Fore.GREEN + f"{index_path} was found") if os.path.exists(index_path) else print(Fore.RED + f"{index_path} was not found")
pitch = 0 # @param {type:"slider", min:-12, max:12, step:1}
input_path = ""#@param {type:"string"}
if not os.path.exists(input_path):
    raise ValueError(f"{input_path} was not found in your RVC folder.")
os.environ['index_root']  = os.path.dirname(index_path)
index_path = os.path.basename(index_path)
f0_method = "rmvpe" # @param ["rmvpe", "pm", "harvest"]
save_as = " /content/KiwKiw/audios/cli_output.wav"#@param {type:"string"}
model_name = os.path.basename(model_path)
os.environ['weight_root'] = os.path.dirname(model_path)
index_rate = 0.5 # @param {type:"slider", min:0, max:1, step:0.01}
volume_normalization = 0 #param {type:"slider", min:0, max:1, step:0.01}
consonant_protection = 0.5 #param {type:"slider", min:0, max:1, step:0.01}

!rm -f $save_as

!python tools/cmd/infer_cli.py --f0up_key $pitch --input_path $input_path --index_path $index_path --f0method $f0_method --opt_path $save_as --model_name $model_name --index_rate $index_rate --device "cuda:0" --is_half True --filter_radius 3 --resample_sr 0 --rms_mix_rate $volume_normalization --protect $consonant_protection

show_errors = False #@param {type:"boolean"}
if not show_errors:
    ipd.clear_output()
ipd.Audio(save_as)

# Extra

In [ ]:
#@title Download Youtube  Wav acapella

url = "https://youtu.be/mM08Jkne69U?si=w3QTNGF3_A910vIU" #@param {type:"string"}
AUDIO_NAME = "STAY" #@param {type:"string"}

from __future__ import unicode_literals

if dataset == "Drive":
  print("Dataset is set to Drive. Skipping this section")
elif dataset == "Youtube":
  import yt_dlp
  import ffmpeg
  import sys


  ydl_opts = {
      'format': 'bestaudio/best',
  #    'outtmpl': 'output.%(ext)s',
      'postprocessors': [{
          'key': 'FFmpegExtractAudio',
          'preferredcodec': 'wav',
      }],
      "outtmpl": f'youtubeaudio/{AUDIO_NAME}',  # this is where you can edit how you'd like the filenames to be formatted
  }
  def download_from_url(url):
      ydl.download([url])
      # stream = ffmpeg.input('output.m4a')
      # stream = ffmpeg.output(stream, 'output.wav')


  with yt_dlp.YoutubeDL(ydl_opts) as ydl:

        download_from_url(url)